In [5]:
import os
os.chdir('/home/kreffert/Probabilistic_LTSF/BasicTS/')
from basicts.metrics import masked_mae, masked_mse, nll_loss, crps, Evaluator, quantile_loss, empirical_crps
from easytorch.device import set_device_type
from easytorch.utils import get_logger, set_visible_devices
# set the device type (CPU, GPU, or MLU)
device_type ='gpu'
gpus = '0'
set_device_type(device_type)
set_visible_devices(gpus)
from easydict import EasyDict

def load_cfg(cfg, random_state=None):
    from easytorch.config import init_cfg
    # cfg path which start with dot will crash the easytorch, just remove dot
    while isinstance(cfg, str) and cfg.startswith(('./','.\\')):
        cfg = cfg[2:]
    # while ckpt_path.startswith(('./','.\\')):
    #     ckpt_path = ckpt_path[2:]
    
    # initialize the configuration
    cfg = init_cfg(cfg, save=False)
    cfg['METRICS'] = EasyDict()
    all_metrics = [#"MSE", "abs_error", "abs_target_sum", "abs_target_mean",
                    # "MAPE", "sMAPE", "MASE", "RMSE", "NRMSE", "ND", "weighted_ND",
                    "mean_absolute_QuantileLoss", "CRPS", "MAE_Coverage", "NLL", 
                    #"VS", "ES"
                    ]
    cfg['METRICS']['FUNCS'] = EasyDict({
        'NLL': nll_loss,
        'CRPS': crps,
        # 'Evaluator': Evaluator(distribution_type=MODEL_PARAM['distribution_type'], 
        #                        quantiles=MODEL_PARAM['quantiles']),
        'Val_Evaluator': Evaluator(distribution_type=cfg['MODEL']['PARAM']['distribution_type'], metrics = all_metrics,
                                quantiles=cfg['MODEL']['PARAM']['prob_args']['quantiles']),  # only use the evaluator during validation/testing iters
    })

    
    if random_state is not None:
        print(f'Using random state {random_state}')
        import os
        os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
        cfg['ENV'] = EasyDict() # Environment settings. Default: None
        # GPU and random seed settings
        cfg['ENV']['TF32'] = True # Whether to use TensorFloat-32 in GPU. Default: False. See https://pytorch.org/docs/stable/notes/cuda.html#tf32-on-ampere.
        cfg['ENV']['SEED'] = random_state # Random seed. Default: None
        cfg['ENV']['DETERMINISTIC'] = True # Whether to set the random seed to get deterministic results. Default: False
        cfg['ENV']['CUDNN'] = EasyDict()
        cfg['ENV']['CUDNN']['ENABLED'] = True # Whether to enable cuDNN. Default: True
        cfg['ENV']['CUDNN']['BENCHMARK'] = True# Whether to enable cuDNN benchmark. Default: True
        cfg['ENV']['CUDNN']['DETERMINISTIC'] = False # Whether to set cuDNN to deterministic mode. Default: False
    return cfg

def evaluate_configs(configs):
    for key in configs.keys():
        configs[key]['cfg'] = load_cfg(configs[key]['cfg'], random_state=None)
        cfg = configs[key]['cfg']
        print(cfg)
        ckpt_path = '/home/kreffert/Probabilistic_LTSF/BasicTS/' + configs[key]['ckpt']
        strict = True
        runner = cfg['RUNNER'](cfg)
        # setup the graph if needed
        if runner.need_setup_graph:
            runner.setup_graph(cfg=cfg, train=False)
            
        print(f'Loading model checkpoint from {ckpt_path}')
        runner.load_model(ckpt_path=ckpt_path, strict=strict)
        
        runner.test_pipeline(cfg=cfg, save_metrics=False, save_results=False)

2025-06-05 21:55:12,004 - easytorch-env - INFO - Use devices 0.


In [6]:
# 1. load the model and set the device
configs = {#'ETTh1_PTST_u': {'cfg':'final_weights/PatchTST/univariate/ETTh1_prob.py',
           #                 'ckpt': 'final_weights/PatchTST/univariate/ETTh1_100_96_720/a8de06edad7530010e0b704422b431a2/PatchTST_best_val_NLL.pt'
           #                },
           'ETTh1_PTST_q': {'cfg': 'final_weights/PatchTST/quantile/ETTh1_prob.py',
                            'ckpt': 'final_weights/PatchTST/quantile/ETTh1_100_96_720/a2a39ac1680165e5ffbda2c7bbda5add/PatchTST_best_val_QL.pt'
                           }
          }
evaluate_configs(configs)

KeyError: 'quantiles'